In [26]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D, Dropout
from keras.layers.advanced_activations import PReLU
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
K.set_image_dim_ordering('th')

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 0.2
    return K.mean(y_true * K.square(y_pred) +
                  (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

convnet = Sequential()

# C1
convnet.add(Conv2D(96,(7,7), activation = 'linear', input_shape = (10,121,53)))
convnet.add(PReLU())
convnet.add(MaxPooling2D(pool_size=(2,2)))
print(convnet.output_shape)

# C2
convnet.add(Conv2D(256,(5,5), activation = 'linear'))
convnet.add(PReLU())
convnet.add(MaxPooling2D(pool_size=(2,2)))
print(convnet.output_shape)

# C3
convnet.add(Conv2D(256,(5,5), activation = 'linear'))
convnet.add(PReLU())
convnet.add(MaxPooling2D(pool_size=(2,2)))
print(convnet.output_shape)

# F4
convnet.add(Flatten())
convnet.add(Dense(4096, activation = 'linear'))
convnet.add(PReLU())
convnet.add(Dropout(0.5))
print(convnet.output_shape)

# F5
convnet.add(Dense(1024, activation = 'linear'))
convnet.add(PReLU())
convnet.add(Dropout(0.5))
print(convnet.output_shape)

# F6
convnet.add(Dense(512, activation = 'linear'))
convnet.add(PReLU())
convnet.add(Dropout(0.5))
print(convnet.output_shape)

# Softmax 
convnet.add(Dense(1, activation='softmax'))
print(convnet.output_shape)

convnet.compile(loss = contrastive_loss,
             optimizer = 'adam',
             metrics=['accuracy'])



(None, 96, 57, 23)
(None, 256, 26, 9)
(None, 256, 11, 2)
(None, 4096)
(None, 1024)
(None, 512)
(None, 1)
